# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa9a52d2df0>
├── 'a' --> tensor([[-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047]])
└── 'x' --> <FastTreeValue 0x7fa9a5258610>
    └── 'c' --> tensor([[-1.8672,  0.1021, -0.1309,  1.1562],
                        [-0.0306,  1.1050,  0.7210, -0.3072],
                        [-0.1514,  0.1227,  0.5167,  0.5828]])

In [4]:
t.a

tensor([[-0.0099, -1.1736,  0.4323],
        [-0.9962,  0.3746, -0.6047]])

In [5]:
%timeit t.a

73.8 ns ± 0.0388 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa9a52d2df0>
├── 'a' --> tensor([[ 1.8116,  0.2079, -1.3023],
│                   [-0.7126,  0.9680,  0.2536]])
└── 'x' --> <FastTreeValue 0x7fa9a5258610>
    └── 'c' --> tensor([[-1.8672,  0.1021, -0.1309,  1.1562],
                        [-0.0306,  1.1050,  0.7210, -0.3072],
                        [-0.1514,  0.1227,  0.5167,  0.5828]])

In [7]:
%timeit t.a = new_value

75.2 ns ± 0.0419 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0099, -1.1736,  0.4323],
               [-0.9962,  0.3746, -0.6047]]),
    x: Batch(
           c: tensor([[-1.8672,  0.1021, -0.1309,  1.1562],
                      [-0.0306,  1.1050,  0.7210, -0.3072],
                      [-0.1514,  0.1227,  0.5167,  0.5828]]),
       ),
)

In [10]:
b.a

tensor([[-0.0099, -1.1736,  0.4323],
        [-0.9962,  0.3746, -0.6047]])

In [11]:
%timeit b.a

65.1 ns ± 0.182 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3121,  0.8102, -2.3526],
               [-0.5091,  0.6381, -0.6269]]),
    x: Batch(
           c: tensor([[-1.8672,  0.1021, -0.1309,  1.1562],
                      [-0.0306,  1.1050,  0.7210, -0.3072],
                      [-0.1514,  0.1227,  0.5167,  0.5828]]),
       ),
)

In [13]:
%timeit b.a = new_value

547 ns ± 0.231 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

878 ns ± 0.438 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 906 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 542 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa8d53efee0>
├── 'a' --> tensor([[[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]],
│           
│                   [[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]],
│           
│                   [[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]],
│           
│                   [[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]],
│           
│                   [[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]],
│           
│                   [[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]],
│           
│                   [[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]],
│           
│                   [[-0.0099, -1.1736,  0.4323],
│                    [-0.9962,  0.3746, -0.6047]]])
└── 'x' --> <FastTreeValue 0x7fa9a52ba850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 81 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa8d53ef8e0>
├── 'a' --> tensor([[-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047],
│                   [-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047],
│                   [-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047],
│                   [-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047],
│                   [-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047],
│                   [-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047],
│                   [-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047],
│                   [-0.0099, -1.1736,  0.4323],
│                   [-0.9962,  0.3746, -0.6047]])
└── 'x' --> <FastTreeValue 0x7fa8d53efbb0>
    └── 'c' --> tensor([[-1.8672,  0.1021, -0.1309,  1.1562],
                        [-0.0306,  1.1050,  0.7210, -0.3072],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 85.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.7 µs ± 58.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.8672,  0.1021, -0.1309,  1.1562],
                       [-0.0306,  1.1050,  0.7210, -0.3072],
                       [-0.1514,  0.1227,  0.5167,  0.5828]],
              
                      [[-1.8672,  0.1021, -0.1309,  1.1562],
                       [-0.0306,  1.1050,  0.7210, -0.3072],
                       [-0.1514,  0.1227,  0.5167,  0.5828]],
              
                      [[-1.8672,  0.1021, -0.1309,  1.1562],
                       [-0.0306,  1.1050,  0.7210, -0.3072],
                       [-0.1514,  0.1227,  0.5167,  0.5828]],
              
                      [[-1.8672,  0.1021, -0.1309,  1.1562],
                       [-0.0306,  1.1050,  0.7210, -0.3072],
                       [-0.1514,  0.1227,  0.5167,  0.5828]],
              
                      [[-1.8672,  0.1021, -0.1309,  1.1562],
                       [-0.0306,  1.1050,  0.7210, -0.3072],
                       [-0.1514,  0.1227,  0.5167,  0.5828]],

In [26]:
%timeit Batch.stack(batches)

83.5 µs ± 99.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.8672,  0.1021, -0.1309,  1.1562],
                      [-0.0306,  1.1050,  0.7210, -0.3072],
                      [-0.1514,  0.1227,  0.5167,  0.5828],
                      [-1.8672,  0.1021, -0.1309,  1.1562],
                      [-0.0306,  1.1050,  0.7210, -0.3072],
                      [-0.1514,  0.1227,  0.5167,  0.5828],
                      [-1.8672,  0.1021, -0.1309,  1.1562],
                      [-0.0306,  1.1050,  0.7210, -0.3072],
                      [-0.1514,  0.1227,  0.5167,  0.5828],
                      [-1.8672,  0.1021, -0.1309,  1.1562],
                      [-0.0306,  1.1050,  0.7210, -0.3072],
                      [-0.1514,  0.1227,  0.5167,  0.5828],
                      [-1.8672,  0.1021, -0.1309,  1.1562],
                      [-0.0306,  1.1050,  0.7210, -0.3072],
                      [-0.1514,  0.1227,  0.5167,  0.5828],
                      [-1.8672,  0.1021, -0.1309,  1.1562],
                   

In [28]:
%timeit Batch.cat(batches)

154 µs ± 273 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

349 µs ± 4.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
